In [1]:
import imodels
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from imodels.tree.rf_plus.rf_plus.rf_plus_models import RandomForestPlusRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, mean_squared_error, r2_score
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import *
from sklearn.preprocessing import StandardScaler
import copy
import matplotlib.pyplot as plt
import openml
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('.')
sys.path.append('./scripts')
from competing_methods_local import *
from simulations_util import *
from collections import Counter
from ucimlrepo import fetch_ucirepo

/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
X_df = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_361071.csv')
print(X_df.shape)

(57580, 54)


In [37]:
y_df

,binaryClass
0,N
1,N
2,N
3,N
4,N
...,...
13483,P
13484,P
13485,P
13486,P


In [38]:
y_df['binaryClass'] = y_df['binaryClass'].map({'N': 0, 'P': 1})

In [40]:
y_df.to_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/y_361063.csv', index=False)

In [30]:
# encode the target variable as 0 and 1
y = y_df['class']
y = (y == 'pos').astype(int)

KeyError: 'class'

In [11]:
pd.set_option('display.max_columns', None)

In [26]:
y= sample_real_data_y(source="openml", task_id="361062", return_support=False)

In [27]:
y

array(['N', 'N', 'N', ..., 'P', 'P', 'P'], dtype=object)

In [25]:
y.value_counts()

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [19]:
y.astype(np.float32)

AttributeError: 'tuple' object has no attribute 'astype'

In [2]:
temp_df = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_9978.csv')

In [10]:
# check if all values in temp_df are numeric
for col in temp_df.columns:
    if not pd.api.types.is_numeric_dtype(temp_df[col]):
        print(col)
        print(temp_df[col].unique())

In [3]:
X_361070 = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_9978_original.csv')

In [ ]:
threshold = 0.99  # Adjust to 0.95 if needed

# Compute the correlation matrix
correlation_matrix = X_361070.corr()

# Identify features with high correlation
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
high_correlation_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

# Drop highly correlated features
X_361070_reduced = X_361070.drop(columns=high_correlation_features)

print(f"Original number of features: {X_361070.shape[1]}")
print(f"Reduced number of features: {X_361070_reduced.shape[1]}")

Original number of features: 72
Reduced number of features: 47


In [87]:
X_361070_reduced.to_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_9978.csv', index=False)

In [3]:
selected_id = []
for task_id in [31, 10101, 3913, 3, 3917, 9957, 9946, 3918, 3903, 37, 9971, 9952, 3902,
                49, 43, 9978, 10093, 219, 9976, 14965, 6, 9977, 53, 11, 15, 16, 14,
                32, 3549, 12, 9981, 18, 28, 2074, 29, 45, 125922, 9960, 9964, 22, 2079,
                14969, 3560, 14952, 125920, 23, 3904, 3022, 9985, 9910, 14970, 3021, 
                3481, 7592, 3573, 146824, 146820, 146822, 146195, 146800, 146817, 146819,
                146821, 167119, 14954, 167141, 167140, 167120, 167125, 146825, 167124, 167121]:
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X = pd.DataFrame(X, columns=attribute_names[:-1])
    if X.shape[1] > 15 and y.nunique() == 2:
        selected_id.append(task_id)

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/task.py:150: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  return datasets.get_dataset(self.dataset_id)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/

In [4]:
selected_id

[31,
 3913,
 3,
 3917,
 9957,
 9946,
 3918,
 3903,
 3902,
 43,
 9978,
 9976,
 9977,
 14952,
 3904,
 9910,
 3021,
 146819,
 14954,
 167141,
 167120,
 167125]

In [15]:
task_id

167121

In [11]:
task = openml.tasks.get_task(361272)
dataset = task.get_dataset()
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

In [12]:
X

,age,height_cm,weight_kg,nationality_name,overall,potential,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,...,movement_agility,movement_reactions,movement_balance,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,34,170,72,Argentina,93,93,85,95,70,91,...,91,94,95,35,24,6,11,15,14,8
1,32,185,81,Poland,92,92,71,95,90,85,...,77,93,82,42,19,15,6,12,8,10
2,36,187,83,Portugal,91,91,87,95,90,80,...,86,94,74,32,24,7,11,15,14,11
3,29,175,68,Brazil,91,91,85,83,63,86,...,96,89,84,32,29,9,9,15,15,11
4,30,181,70,Belgium,91,91,94,82,55,94,...,79,91,78,65,53,15,13,5,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19173,22,180,64,China PR,47,52,46,32,48,50,...,55,53,70,43,48,6,10,5,15,13
19174,19,175,70,Republic of Ireland,47,59,54,33,46,51,...,64,49,69,44,47,11,12,6,8,10
19175,21,178,72,Republic of Ireland,47,55,39,32,43,49,...,58,46,59,44,48,8,6,7,10,6
19176,19,173,66,Republic of Ireland,47,60,29,49,40,38,...,72,48,73,14,11,7,10,7,14,15


In [7]:
# encode category to 1/0 in X
X = pd.get_dummies(X)
X = X.astype(float)

In [8]:
X

,bkblk_t,bkblk_f,bknwy_t,bknwy_f,bkon8_t,bkon8_f,bkona_t,bkona_f,bkspr_t,bkspr_f,...,wkna8_f,wknck_t,wknck_f,wkovl_t,wkovl_f,wkpos_t,wkpos_f,wtoeg_n,wtoeg_t,wtoeg_f
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3191,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3192,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3193,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3194,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [2]:
for iid in [43, 361062, 361071, 9978, 361069, 361068, 361063, 361622, 361260, 361259, 361253, 361254, 361242, 361243]:
    task = openml.tasks.get_task(iid)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X_df = pd.DataFrame(X)
    X_df.to_csv(f"/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_{iid}.csv", index=False)
    y_df = pd.DataFrame(y)
    y_df.to_csv(f"/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/y_{iid}.csv", index=False)

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

In [3]:
X.shape[1]

116

In [32]:
# selected_id = []
# for task_id in [31, 10101, 3913, 3, 3917, 9957, 9946, 3918, 3903, 37, 9971, 9952, 3902,
#                 49, 43, 9978, 10093, 219, 9976, 14965, 6, 9977, 53, 11, 15, 16, 14,
#                 32, 3549, 12, 9981, 18, 28, 2074, 29, 45, 125922, 9960, 9964, 22, 2079,
#                 14969, 3560, 14952, 125920, 23, 3904, 3022, 9985, 9910, 14970, 3021, 
#                 3481, 7592, 3573, 146824, 146820, 146822, 146195, 146800, 146817, 146819,
#                 146821, 167119, 14954, 167141, 167140, 167120, 167125, 146825, 167124, 167121]:
#     task = openml.tasks.get_task(task_id)
#     dataset = task.get_dataset()
#     X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
#     X = pd.DataFrame(X, columns=attribute_names[:-1])
#     if X.shape[1] > 10 and y.nunique() == 2:
#         selected_id.append(task_id)

selected_id = []
for task_id in [361055, 361064, 361056, 361057, 361071, 361067, 361063, 361065, 361066, 361060, 361068,
                361062, 361069, 361061, 361070]:
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X = pd.DataFrame(X, columns=attribute_names[:-1])
    if X.shape[1] > 10 and y.nunique() == 2:
        selected_id.append(task_id)

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

In [9]:
task_id = 361079
task = openml.tasks.get_task(task_id)
dataset = task.get_dataset()

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

In [10]:
X, _, _, _ = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")
X.shape

/tmp/ipykernel_1417537/276232148.py:1: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` function to continue using numpy arrays.
  X, _, _, _ = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")


(13750, 33)

In [13]:
for task_id in [361091]: #[361072, 361073, 361074, 361077, 361079, 361084, 361088, 361079]:
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X_df = pd.DataFrame(X)
    print(X_df.shape)
    X_df.to_csv(f"/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_{task_id}.csv", index=False)
    
    y_df = pd.DataFrame(y)
    print(y_df.shape)
    y_df.to_csv(f"/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/y_{task_id}.csv", index=False)

KeyboardInterrupt: 

In [55]:
X, _, _, _ = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")
X_df = pd.DataFrame(X)
X_df.to_csv(f"/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_{task_id}.csv", index=False)

/tmp/ipykernel_3921584/2916645993.py:1: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` function to continue using numpy arrays.
  X, _, _, _ = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")


In [56]:
task = openml.tasks.get_task(task_id)
dataset = task.get_dataset()
_, y, _, _ = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")
y_df = pd.DataFrame(y)
y_df.to_csv(f"/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/y_{task_id}.csv", index=False)

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

In [43]:
X_df.shape

(5188, 20)

In [25]:
X

,lineNo,assgNo,prevFixDur,firstfixDur,firstPassFixDur,nextFixDur,firstSaccLen,lastSaccLen,prevFixPos,landingPos,leavingPos,totalFixDur,meanFixDur,regressLen,regressDur,pupilDiamMax,pupilDiamLag,timePrtctg,titleNo,wordNo
0,9684.0,298.0,99.0,119.0,119.0,238.0,300.1666,288.0156,246.0732,56.6326,51.1786,119.0,119.0000,0.0,0.0,0.1541,0.4446,0.0176,4,3
1,2065.0,66.0,119.0,159.0,239.0,100.0,306.1066,117.2359,321.0140,47.3207,47.0425,239.0,119.5000,0.0,159.0,0.0119,0.3264,0.0239,8,1
2,1969.0,61.0,139.0,139.0,357.0,179.0,590.5294,805.0531,479.3078,112.3610,118.6529,536.0,178.6667,258.0,0.0,0.0874,0.0859,0.0431,4,8
3,10326.0,315.0,199.0,139.0,219.0,99.0,80.9475,306.5587,153.1698,79.8389,78.5875,198.0,99.0000,0.0,396.0,0.0173,0.1453,0.0264,4,2
4,6827.0,204.0,80.0,100.0,239.0,139.0,110.7576,161.4845,189.6207,96.7471,102.1225,139.0,139.0000,0.0,378.0,0.3229,0.4074,0.0288,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7603,10923.0,336.0,219.0,219.0,219.0,119.0,277.4140,27.8568,281.0640,18.6078,4.4721,219.0,219.0000,417.0,139.0,0.2830,0.4085,0.0187,9,4
7604,10924.0,336.0,219.0,119.0,278.0,139.0,27.8568,69.0290,111.3643,84.8528,71.9809,278.0,139.0000,0.0,278.0,0.2830,0.4730,0.0237,9,3
7605,10925.0,336.0,219.0,219.0,219.0,119.0,69.0290,165.0273,55.7136,30.3026,26.2011,139.0,139.0000,417.0,139.0,0.2830,0.4730,0.0119,9,4
7606,10926.0,336.0,139.0,179.0,179.0,80.0,165.0273,1445.4747,122.5887,38.3471,1343.5316,179.0,179.0000,0.0,0.0,0.2230,0.4730,0.0153,9,5


In [ ]:
# [31,
#  3913,
#  3,
#  3917,
#  9957,
#  9946,
#  3918,
#  3903,
#  3902,
#  43,
#  9978,
#  9976,
#  14965,
#  9977,
#  29,
#  14952,
#  125920,
#  3904,
#  9910,
#  3021,
#  7592,
#  146819,
#  14954,
#  167141,
#  167120,
#  167125]

In [3]:
def check_columns_with_few_distinct_values(df):
    """
    Check if any column in the DataFrame has fewer than 10 distinct values.

    Parameters:
    df (pd.DataFrame): The input DataFrame.

    Returns:
    dict: A dictionary with column names as keys and the count of distinct values as values for columns with fewer than 10 distinct values.
    """
    result = {col: df[col].nunique() for col in df.columns if df[col].nunique() < 10}
    return result

for t_id in [361256, 361272, 361268, 361243, 361266, 361619]:
    task = openml.tasks.get_task(t_id)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X = pd.DataFrame(X, columns=attribute_names[:-1])
    print(t_id)
    print(check_columns_with_few_distinct_values(X))

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/py

361256
{}


/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/task.py:150: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_d

361272
{}


/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/task.py:150: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_d

...

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/task.py:150: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_d

361266
{'floors': 6, 'waterfront': 2, 'view': 5, 'condition': 5, 'date_year': 2}


/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)


361619
{'school': 2, 'sex': 2, 'age': 8, 'address': 2, 'famsize': 2, 'Pstatus': 2, 'Medu': 5, 'Fedu': 5, 'Mjob': 5, 'Fjob': 5, 'reason': 4, 'guardian': 3, 'traveltime': 4, 'studytime': 4, 'failures': 4, 'schoolsup': 2, 'famsup': 2, 'paid': 2, 'activities': 2, 'nursery': 2, 'higher': 2, 'internet': 2, 'romantic': 2, 'famrel': 5, 'freetime': 5, 'goout': 5, 'Dalc': 5, 'Walc': 5, 'health': 5}


/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/task.py:150: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  return datasets.get_dataset(self.dataset_id)


In [ ]:
selected_id

In [ ]:
# a lot of 361064 361068

In [ ]:
task = openml.tasks.get_task(361070) #24 21
dataset = task.get_dataset()
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
X

In [ ]:
X.corr()

In [ ]:
# check if there are NA in X
X.isna().sum()

In [ ]:
y

In [ ]:
y

In [ ]:
#Final selected ids
# task_ids = [3917, 9946, 43, 9978, 146819, 167120]

#3904 has missing

In [ ]:
X = sample_real_data_X(source="openml", task_id = 9946, normalize=True) #361243
y = sample_real_data_y(source="openml", task_id = 9946)[0] #361243


# partial_linear_lss_model_random_feature(X, beta=1, heritability=0.4, sigma=None, s=5, error_seed=1, feature_seed=1, return_support=True)

In [ ]:
# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
prediction_model_ridge=RidgeCV(cv=5)
prediction_model_lasso=LassoCV(cv=5)
prediction_model_en=ElasticNetCV(cv=5)
prediction_model_ridge.fit(X_train, y_train)
prediction_model_lasso.fit(X_train, y_train)
prediction_model_en.fit(X_train, y_train)

In [ ]:
import numpy as np
from sklearn.linear_model import RidgeCV, ElasticNetCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Generate synthetic data for demonstration
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Fit RidgeCV
ridge = RidgeCV(cv=5, scoring='neg_mean_squared_error')
ridge.fit(X_train, y_train)
ridge_cv_error = -1*ridge.best_score_

# Fit LassoCV
lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_train, y_train)
lasso_cv_error = np.mean(lasso.mse_path_, axis=1).min()

# Fit ElasticNetCV
elasticnet = ElasticNetCV(cv=5,l1_ratio=[0.1,0.5,0.7,0.9,0.95,0.99,1], random_state=0) #1.0 mean Lasso
elasticnet.fit(X_train, y_train)
elasticnet_cv_error = np.mean(elasticnet.mse_path_, axis=-1).min()

# Select the best model
errors = {
    'Ridge': ridge_cv_error,
    'Lasso': lasso_cv_error,
    'ElasticNet': elasticnet_cv_error
}
best_model_name = min(errors, key=errors.get)
best_model = {'Ridge': ridge, 'Lasso': lasso, 'ElasticNet': elasticnet}[best_model_name]

# Output results
print(f"Cross-validation errors: {errors}")
print(f"Best model: {best_model_name}")


In [ ]:
# compute ridge mse on training
y_pred_ridge = ridge.predict(X_train)
ridge_mse = mean_squared_error(y_train, y_pred_ridge)
print(f"Ridge MSE on training: {ridge_mse}")

In [ ]:
ridge.alpha_

In [ ]:
y_pred = ridge.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE of the best model ({best_model_name}): {test_mse}")


In [ ]:
y_pred = lasso.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE of the best model ({best_model_name}): {test_mse}")

In [ ]:
y_pred = elasticnet.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE of the best model ({best_model_name}): {test_mse}")


In [ ]:
elasticnet.mse_path_

In [ ]:
ridge

In [ ]:
elasticnet.mse_path_.shape

In [ ]:
np.mean(-elasticnet.mse_path_, axis=1)

In [ ]:
prediction_model_ridge.mse_path_

In [ ]:
prediction_model_ridge.best_score_

In [ ]:
from sklearn.linear_model import LassoCV
model = LassoCV(cv=5, max_iter=10000)
model.fit(X_train, y_train)

best_alpha_idx = np.where(model.alphas_ == model.alpha_)[0][0]

cv_errors = model.mse_path_
# Get the cross-validation errors for the best alpha
errors_for_best_alpha = cv_errors[best_alpha_idx, :]

# Compute the mean cross-validation error for the best alpha
mean_error_for_best_alpha = errors_for_best_alpha.mean()

print("Mean CV Error for the Best Alpha:", mean_error_for_best_alpha)
print(np.mean(model.mse_path_, axis=1).min())

In [ ]:
from sklearn.linear_model import ElasticNetCV
model = ElasticNetCV(cv=5,l1_ratio=[0.1, 0.5, 0.7, 0.9], max_iter=10000)
model.fit(X_train, y_train)
np.mean(model.mse_path_, axis=-1).min()

In [ ]:
np.mean(model.mse_path_, axis=-1).min()

In [ ]:
model =LogisticRegressionCV(penalty='l1', solver = 'saga', cv=5, max_iter=10000)
model.fit(X_train, y_train)
print(np.mean(model.scores_[1], axis=0).max())

In [ ]:
model.scores_[1]

In [ ]:
# compute accuracy on the training set
y_train_pred = model.predict(X_train)
accuracy_train = np.mean(y_train == y_train_pred)
print('Training Accuracy:', accuracy_train)

In [ ]:
model =LogisticRegressionCV(penalty='l2', cv=5, max_iter=10000)
model.fit(X_train, y_train)
print(np.mean(model.scores_[1], axis=0).max())

In [ ]:
model =LogisticRegressionCV(penalty='elasticnet',l1_ratios=[0.1, 0.5, 0.7, 0.9], solver = 'saga', cv=5, max_iter=10000)
model.fit(X_train, y_train)
print(np.mean(model.scores_[1], axis=0).max())

In [ ]:
np.mean(model.scores_[1], axis=0).max()

In [ ]:
np.mean(model.scores_[1], axis=0)

In [ ]:
model.Cs_

In [ ]:
model.C_

In [ ]:
optimal_c_index = np.where(model.Cs_ == model.C_[0])[0][0]  # Optimal C value index
optimal_l1_index = np.argmax([score.mean() for score in model.scores_[1]])  # Optimal l1_ratio index

# Compute the mean cross-validated score for the optimal C and l1_ratio
mean_cv_score_optimal = model.scores_[1][optimal_l1_index][:, optimal_c_index].mean()

# Print results
print(f"Mean cross-validated score (optimal): {mean_cv_score_optimal}")
print(f"Test set score: {model.score(X_test, y_test)}")

In [ ]:
optimal_l1_index

In [ ]:
model.Cs_

In [ ]:
model.C_

In [ ]:
model.scores_[1]

In [ ]:
model =LogisticRegressionCV(penalty='l1', solver = 'saga', cv=5, max_iter=10000)
model.fit(X, y)
optimal_c_index = np.where(model.Cs_ == model.C_[0])[0][0]
mean_cv_score_optimal = model.scores_[1][:, optimal_c_index].mean()
print(mean_cv_score_optimal)

In [ ]:
optimal_c_index = np.where(model.Cs_ == model.C_[0])[0][0]
mean_cv_score_optimal = model.scores_[1][:, optimal_c_index].mean()

In [ ]:
model.Cs_

In [ ]:
# 'scores_' is a dictionary where keys are the classes and values are arrays of cross-validation scores
cv_scores = model.scores_

# For binary classification, get scores for the positive class (assumes classes are 0 and 1)
positive_class_scores = cv_scores[1]  # Replace '1' with the appropriate class if needed

# Compute mean cross-validation score for each fold
mean_cv_scores = positive_class_scores.mean(axis=0)
print("Mean CV scores for each fold:", mean_cv_scores)

In [ ]:
model.coefs_paths_

In [ ]:
ridge.cv_values_

In [ ]:
ridge.alpha_

In [ ]:
assert False

In [ ]:
y[15]

In [ ]:
0.242974+0.33677068-1.2268368+2

In [ ]:
support

In [ ]:
X[0]

In [ ]:
0.5324-0.404-0.5635

In [ ]:
y[0]

In [ ]:
from sklearn.linear_model import LassoCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf.fit(X_train, y_train)
rf_plus_base_lasso = RandomForestPlusRegressor(rf_model=rf, prediction_model=LassoCV(cv=5, max_iter=8000))
rf_plus_base_lasso.fit(X_train, y_train)

In [ ]:
rf_plus_alo_mdi = RFPlusMDI(rf_plus_base_lasso, mode = 'only_k', evaluate_on="all")
local_fi_score_train = rf_plus_alo_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)
local_fi_score_bootstrap_train = rf_plus_alo_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True, bootstrap=True)

In [ ]:
local_fi_score_train[4]

In [ ]:
roc_auc_score(support, local_fi_score_train[4])

In [ ]:
local_fi_score_bootstrap_train.shape

In [ ]:
local_fi_score_bootstrap_train[150, :,:].shape

In [ ]:
roc_auc_score(support, np.mean(np.argsort(np.argsort(np.abs(local_fi_score_bootstrap_train[10, :,:]), axis=0), axis=0), axis=1))

In [ ]:
np.argsort(np.argsort(np.abs(local_fi_score_bootstrap_train[150, :,:])))

In [ ]:
X.shape

In [ ]:
np.mean(local_fi_score_bootstrap_train[2, :,:], axis=-1)

In [ ]:
treeshap_train, _ = tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf, mode="absolute")

In [ ]:
auroc = []
for i in range(local_fi_score_train.shape[0]):
    fi_data_i = local_fi_score_train[i]    
    auroc.append(roc_auc_score(support, fi_data_i))
print(np.mean(auroc))

In [ ]:
auroc_shap = []
for i in range(treeshap_train.shape[0]):
    fi_data_i = treeshap_train[i]   
    auroc_shap.append(roc_auc_score(support, fi_data_i))
print(np.mean(auroc_shap))

In [ ]:
auroc_bootstrap = []
for i in range(local_fi_score_bootstrap_train.shape[0]):
    fi_data_i = local_fi_score_bootstrap_train[i]    
    auroc_bootstrap.append(roc_auc_score(support, fi_data_i))
print(np.mean(auroc_bootstrap))

In [ ]:
auroc_bootstrap[4]

In [ ]:
auroc[4]

In [ ]:
auroc_bootstrap = np.array(auroc_bootstrap)
auroc = np.array(auroc)

indices = auroc_bootstrap > auroc
gains = auroc_bootstrap[indices] - auroc[indices]
print("Gains:", sum(gains))
print("Number of gains:", sum(indices))

In [ ]:
indices = auroc_bootstrap < auroc
gains = auroc_bootstrap[indices] - auroc[indices]
print("Gains:", sum(gains))
print("Number of gains:", sum(indices))

In [ ]:
gains[4]

In [ ]:
auroc_bootstrap[4]

In [ ]:
auroc[4]

In [ ]:
np.argsort(-1*support)

In [ ]:
np.argsort(-1*local_fi_score_train[4])

In [ ]:
np.argsort(-1*local_fi_score_bootstrap_train[4])

In [ ]:
indices = auroc_bootstrap ==  auroc
gains = auroc_bootstrap[indices] - auroc[indices]
print("Gains:", sum(gains))
print("Number of gains:", sum(indices))

In [ ]:
sum(auroc)

In [ ]:
sum(auroc_bootstrap)

In [ ]:
print(sum(np.array(auroc_bootstrap) < np.array(auroc)))

In [ ]:
for i in range(local_fi_score_train.shape[0]):
    if auroc_bootstrap[i] > auroc[i]:
        print(i)
        print(auroc_bootstrap[i] - auroc[i])

In [ ]:
assert False

In [ ]:
np.abs(rf_plus.estimators_[0].coef_)[:20]

In [ ]:
rf_plus.estimators_[0].loo_coefficients_[:, :-1][0][:20]

In [ ]:
np.argsort(-1*np.argsort(np.argsort(np.abs(rf_plus.estimators_[0].coef_))))

In [ ]:
np.argsort(-1*np.argsort(np.argsort(np.abs(rf_plus.estimators_[0].loo_coefficients_[:, :-1][2]))))

In [ ]:
sum = 0
for i in range(X_train.shape[0]):
    sort_coef = np.argsort(-1 * np.argsort(np.argsort(np.abs(rf_plus.estimators_[0].coef_))))
    sort_loo_coefficients = np.argsort(-1 * np.argsort(np.argsort(np.abs(rf_plus.estimators_[0].loo_coefficients_[:, :-1][i]))))
    if np.array_equal(sort_coef[:5], sort_loo_coefficients[:5]):
        sum += 1
print(sum)


In [ ]:
assert False

In [ ]:
rf_plus.estimators_[0].loo_coefficients_[:, :-1][0]

In [ ]:
np.argsort(np.abs(rf_plus.estimators_[0].coef_[:, :-1][0]))

In [ ]:
np.argsort(np.abs(rf_plus.estimators_[0].loo_coefficients_[:, :-1][0]))

In [ ]:
rf_plus_alo_mdi = AloRFPlusMDI(rf_plus, mode = 'only_k', evaluate_on="all")
local_fi_score_train = rf_plus_alo_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

In [ ]:
rf_plus_mdi = RFPlusMDI(rf_plus, mode = 'only_k', evaluate_on="all")
local_fi_score_train = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

In [ ]:
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True, bootstrap=True)

In [ ]:
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True, bootstrap=False)

In [ ]:
local_fi_score_train

In [ ]:
#plot y
plt.hist(y)
plt.show()

In [ ]:
y = sample_real_data_y(source="openml", task_id = 361247)

In [ ]:
y

In [ ]:
"openml_361242" #"openml_361243" #"openml_361247" #"openml_361251" "openml_361253" "openml_361254" "openml_361256" "openml_361259" "openml_361260""openml_361622"
for task_id in [361242, 361243, 361247, 361251, 361253, 361254, 361256, 361259, 361260, 361622]:
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X = pd.DataFrame(X, columns=attribute_names[:-1])
    print(task_id)
    if X.shape[0] > 2000:
        print(int(2000 * 0.66))
    else:
        print(int(X.shape[0]*0.66))

In [ ]:
# for task_id in [361236, 361237, 361235, 361234, 361247, 361243, 361242, 361241, 361244, 361252, 361251, 361249, 361250, 361255, 361253, 361260, 
#                 361259, 361256, 361257, 361261, 361254, 361258, 361264, 361617, 361618, 361269, 361267, 361266, 361268, 361272,
#                 361616, 361619, 361621, 361623, 361622]:
#     task = openml.tasks.get_task(task_id)
#     dataset = task.get_dataset()
#     X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
#     X = pd.DataFrame(X, columns=attribute_names[:-1])
#     if X.shape[1] > 10:
#         print(task_id)
#         print(X.shape[0]*0.66)

In [ ]:
# id with more than 10 features
# selected_id = [361247, 361243, 361242, 361252, 361251, 361253, 361260, 361259, 361256, 361254, 361618, 161266, 361268,
#                361272, 361616, 361619, 361622]
# print(len(selected_id))

In [ ]:
# def check_columns_with_few_distinct_values(df):
#     """
#     Check if any column in the DataFrame has fewer than 10 distinct values.

#     Parameters:
#     df (pd.DataFrame): The input DataFrame.

#     Returns:
#     dict: A dictionary with column names as keys and the count of distinct values as values for columns with fewer than 10 distinct values.
#     """
#     result = {col: df[col].nunique() for col in df.columns if df[col].nunique() < 10}
#     return result

# for t_id in selected_id:
#     task = openml.tasks.get_task(t_id)
#     dataset = task.get_dataset()
#     X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
#     X = pd.DataFrame(X, columns=attribute_names[:-1])
#     print(t_id)
#     print(check_columns_with_few_distinct_values(X))

In [ ]:
# id with no categorical features
selected_id = [361247, 361243, 361242, 361251, 361253, 361260, 361259, 361256, 361254, 361622]
print(len(selected_id))

In [ ]:
# check if there are any missing values
for t_id in selected_id:
    task = openml.tasks.get_task(t_id)
    dataset = task.get_dataset()
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
    X = pd.DataFrame(X, columns=attribute_names[:-1])
    print(t_id)
    print(X.isnull().sum().sum())

# task = openml.tasks.get_task(t_id)
# dataset = task.get_dataset()
# X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")
# # X = pd.DataFrame(X, columns=attribute_names[:-1])
# X

In [ ]:
task = openml.tasks.get_task(361622)
dataset = task.get_dataset()
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")
# X = pd.DataFrame(X, columns=attribute_names[:-1])
y

In [ ]:
np.random.seed(4307)
np.random.choice(X.shape[0], 20, replace=False)

In [ ]:
np.random.seed(4307)
np.random.choice(X.shape[0], 20, replace=False)

In [ ]:
np.random.seed(4307)
np.random.choice(X.shape[0], 20, replace=False)

In [ ]:
y.shape[0]

In [ ]:
np.log(y)

In [ ]:
# Plot y distribution
plt.hist(y)
plt.show()

In [ ]:
y = np.log(y)
plt.hist(y)
plt.show()

In [ ]:
y

In [ ]:
# Turn y into a dataframe
y = pd.DataFrame(y, columns=['target'])

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_no_standardize = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf_no_standardize.fit(X_train, y_train)
rf_plus_base_no_standardize = RandomForestPlusRegressor(rf_model=rf_no_standardize)
rf_plus_base_no_standardize.fit(X_train, y_train)

In [ ]:
assert False

In [ ]:
def preprocessing_data_X(X):
    categorical_cols = X.select_dtypes(include=["object", "category"]).columns
    numerical_cols = X.select_dtypes(include=["number"]).columns
    if X[numerical_cols].isnull().any().any():
        num_imputer = SimpleImputer(strategy="mean")
        X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])
    if len(categorical_cols) > 0 and X[categorical_cols].isnull().any().any():
        # Convert categorical columns to string to ensure consistent types
        X[categorical_cols] = X[categorical_cols].astype(str)
        cat_imputer = SimpleImputer(strategy="most_frequent")
        X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])
    if len(categorical_cols) > 0:
        encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
        X_categorical = encoder.fit_transform(X[categorical_cols])
        X_categorical_df = pd.DataFrame(
            X_categorical,
            columns=encoder.get_feature_names_out(categorical_cols),
            index=X.index
        )
        X = pd.concat([X[numerical_cols], X_categorical_df], axis=1)
    else:
        X = X[numerical_cols]
    X = X.to_numpy()
    if X.shape[0]>2000:
        X = X[:2000,:]
    return X

In [ ]:
# Temperature 925
dataset = fetch_ucirepo(id=857)
X = dataset.data.features
X

In [ ]:
X.select_dtypes(include=["object", "category"]).nunique()

In [ ]:
# Parkinsons 189
dataset = fetch_ucirepo(id=189)
X = dataset.data.features
X

In [ ]:
X.select_dtypes(include=["object", "category"]).nunique()

In [ ]:
X = sample_real_data_X(source="openml", data_id=1245)
y = sample_real_data_y(source="openml", data_id=1245, return_support=False)

In [ ]:
X = sample_real_data_X(source="openml", data_id=1245)
y = sample_real_data_y(source="openml", data_id=1245, return_support=False)

In [ ]:
dataset = openml.datasets.get_dataset(1245)
X, _, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
X = pd.DataFrame(X, columns=attribute_names[:-1])

In [ ]:
X

In [ ]:
X["histology"].value_counts()

In [ ]:
X.select_dtypes(include=["object", "category"]).columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

categorical_cols = X.select_dtypes(include=["object", "category"]).columns
numerical_cols = X.select_dtypes(include=["number"]).columns
if X[numerical_cols].isnull().any().any():
    num_imputer = SimpleImputer(strategy="mean")
    X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])
if len(categorical_cols) > 0 and X[categorical_cols].isnull().any().any():
    X[categorical_cols] = X[categorical_cols].astype(str)
    cat_imputer = SimpleImputer(strategy="most_frequent")
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])
if len(categorical_cols) > 0:
    encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    X_categorical = encoder.fit_transform(X[categorical_cols])
    X_categorical_df = pd.DataFrame(
        X_categorical,
        columns=encoder.get_feature_names_out(categorical_cols),
        index=X.index
    )

In [ ]:
X_categorical_df

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

def preprocessing_data_X(X):
    categorical_cols = X.select_dtypes(include=["object", "category"]).columns
    numerical_cols = X.select_dtypes(include=["number"]).columns
    if X[numerical_cols].isnull().any().any():
        num_imputer = SimpleImputer(strategy="mean")
        X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])
    if len(categorical_cols) > 0 and X[categorical_cols].isnull().any().any():
        X[categorical_cols] = X[categorical_cols].astype(str)
        cat_imputer = SimpleImputer(strategy="most_frequent")
        X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])
    if len(categorical_cols) > 0:
        encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
        X_categorical = encoder.fit_transform(X[categorical_cols])
        X_categorical_df = pd.DataFrame(
            X_categorical,
            columns=encoder.get_feature_names_out(categorical_cols),
            index=X.index
        )
        X = pd.concat([X[numerical_cols], X_categorical_df], axis=1)
    else:
        X = X[numerical_cols]
    X = X.to_numpy()
    if X.shape[0]>2000:
        X = X[:2000,:]
    return X


In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_no_standardize = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf_no_standardize.fit(X_train, y_train)
rf_plus_base_no_standardize = RandomForestPlusRegressor(rf_model=rf_no_standardize)
rf_plus_base_no_standardize.fit(X_train, y_train)

In [ ]:
# print accuracy
print("RF+ accuracy: ", r2_score(y_test, rf_plus_base_no_standardize.predict(X_test)))
print("RF accuracy: ", r2_score(y_test, rf_no_standardize.predict(X_test)))

In [ ]:
shap_score_no_standardize = np.argsort(-1*tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf_no_standardize, mode="absolute")[0])
shap_score_no_standardize 

In [ ]:
lmdi_no_standardize = np.argsort(-1*LFI_evaluation_AloRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_base_no_standardize, mode="absolute")[0])
lmdi_no_standardize

In [ ]:
# standardize data
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf.fit(X_train, y_train)
rf_plus_base = RandomForestPlusRegressor(rf_model=rf)
rf_plus_base.fit(X_train, y_train)
# print accuracy
print("RF+ accuracy: ", r2_score(y_test, rf_plus_base.predict(X_test)))
print("RF accuracy: ", r2_score(y_test, rf.predict(X_test)))

In [ ]:
shap_score = np.argsort(-1*tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf, mode="absolute")[0])
shap_score

In [ ]:
lmdi = np.argsort(-1*LFI_evaluation_AloRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_base, mode="absolute")[0])
lmdi

In [ ]:
# assert 

In [ ]:
np.array_equal(lmdi, lmdi_no_standardize)

In [ ]:
difference_mask = lmdi != lmdi_no_standardize
print("Indices of differing elements:", np.argwhere(difference_mask))

In [ ]:
np.array_equal(shap_score, shap_score_no_standardize)

In [ ]:
difference_mask = shap_score != shap_score_no_standardize
print("Indices of differing elements:", np.argwhere(difference_mask))

In [ ]:
shap_score[5]

In [ ]:
shap_score_no_standardize[5]

In [ ]:
tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf, mode="absolute")[0]

In [ ]:
tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf_no_standardize, mode="absolute")[0]

In [ ]:
# Create heatmap
plt.figure(figsize=(8, 6))
plt.imshow(shap_score_no_standardize, cmap='viridis', aspect='auto')  # Choose colormap as needed
plt.colorbar(label='Values')  # Add a colorbar
plt.title('Matrix Heatmap')  # Add title
plt.xlabel('Columns')  # Add x-axis label
plt.ylabel('Rows')  # Add y-axis label
plt.show()

In [ ]:
# row_tuples = [tuple(row) for row in shap_score]
# row_counts = Counter(row_tuples)
# for row, count in row_counts.items():
#     print(f"Row: {row}, Count: {count}")

In [ ]:
# Create heatmap
plt.figure(figsize=(8, 6))
plt.imshow(lmdi, cmap='viridis', aspect='auto')  # Choose colormap as needed
plt.colorbar(label='Values')  # Add a colorbar
plt.title('Matrix Heatmap')  # Add title
plt.xlabel('Columns')  # Add x-axis label
plt.ylabel('Rows')  # Add y-axis label
plt.show()

In [ ]:
row_tuples = [tuple(row) for row in lmdi]
row_counts = Counter(row_tuples)
for row, count in row_counts.items():
    print(f"Row: {row}, Count: {count}")

In [ ]:
from ucimlrepo import fetch_ucirepo
dataset = fetch_ucirepo(id=10)
dataset.data.targets

In [ ]:
# apply train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf.fit(X_train, y_train)
rf_plus_base = RandomForestPlusRegressor(rf_model=rf)
rf_plus_base.fit(X_train, y_train)

In [ ]:
train_data, test_data = LFI_evaluation_RFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_base, mode="absolute")

In [ ]:
train_data[0]

In [ ]:
np.argsort(-1*train_data[0])[:3].tolist()

In [ ]:
assert False

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example NumPy array
data = LFI_evaluation_RFPlus_all_ranking_retrain(X_train, y_train, fit=rf_plus_base, mode="absolute")
data = np.argsort(data, axis=1)  # Sort the indices of the features

# Adjust the figure size
plt.figure(figsize=(12, 6))  # Width = 12 inches, Height = 6 inches
plt.imshow(data, cmap='viridis', interpolation='nearest', aspect='auto')
plt.colorbar()  # Add a color bar to show the scale
plt.title("Heatmap of NumPy Array")
plt.show()



In [ ]:
LFI_evaluation_RFPlus_all_l2_norm_ranking_retrain(X_train, y_train, fit=rf_plus_base, mode="absolute")

In [ ]:
rf_plus_mdi = RFPlusMDI(rf_plus_base, evaluate_on="all")
temp1 = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)
temp_10 = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, leaf_average=True)
temp2 = rf_plus_mdi.explain_linear_partial_error_metric(X=X_train, y=y_train)
temp3 = rf_plus_mdi.explain_linear_partial_error_metric(X=X_train, y=y_train, leaf_average=True)
temp4 = rf_plus_mdi.explain_linear_partial_error_metric(X=X_train, y=y_train, ranking=True)


In [ ]:
from collections import defaultdict
leaf_indices = rf.apply(X_train).flatten()
leaf_mapping = defaultdict(list)
for sample_idx, leaf_idx in enumerate(leaf_indices):
    leaf_mapping[leaf_idx].append(sample_idx)
leaf_mapping[20]

In [ ]:
temp3[148]

In [ ]:
temp3[82]

In [ ]:
temp1[0]

In [ ]:
(y_train[0] - rf_plus_base.predict(X_train[0].reshape(1, -1)) + temp1[0])**2

In [ ]:
temp2[0]

In [ ]:
temp.shape

In [ ]:
y_train.shape

In [ ]:
result = (temp - y_train[:, np.newaxis, np.newaxis])**2

In [ ]:
y_train[:, np.newaxis, np.newaxis].shape

In [ ]:
y_train[20]

In [ ]:
(0.1676066)**2

In [ ]:
temp[20]

In [ ]:
result[20]

In [ ]:
rf_plus_base = RandomForestPlusRegressor(rf_model=rf)
rf_plus_base.fit(X_train, y_train)
rf_plus_base.score(X_test, y_test)

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
parkinsons_telemonitoring = fetch_ucirepo(id=189) 
  
# data (as pandas dataframes) 
X = parkinsons_telemonitoring.data.features 
y = parkinsons_telemonitoring.data.targets 


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

categorical_cols = X.select_dtypes(include=["object", "category"]).columns
numerical_cols = X.select_dtypes(include=["number"]).columns

# Step 2: Handle missing values (if any)
# Check if there are missing values in the numerical columns
if X[numerical_cols].isnull().any().any():
    # Impute missing values in numerical columns with the mean
    num_imputer = SimpleImputer(strategy="mean")
    X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])

# Check if there are missing values in the categorical columns
if len(categorical_cols) > 0 and X[categorical_cols].isnull().any().any():
    # Convert categorical columns to string to ensure consistent types
    X[categorical_cols] = X[categorical_cols].astype(str)

    # Impute missing values in categorical columns with the most frequent value
    cat_imputer = SimpleImputer(strategy="most_frequent")
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

# Step 3: Encode categorical variables using OneHotEncoder (if any categorical columns)
if len(categorical_cols) > 0:
    encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    X_categorical = encoder.fit_transform(X[categorical_cols])

    # Convert encoded categorical data back to DataFrame
    X_categorical_df = pd.DataFrame(
        X_categorical,
        columns=encoder.get_feature_names_out(categorical_cols),
        index=X.index
    )

    # Step 4: Concatenate numerical columns and the encoded categorical DataFrame
    X = pd.concat([X[numerical_cols], X_categorical_df], axis=1)
else:
    # If no categorical columns, we just use the numerical columns
    X = X[numerical_cols]
X = X.to_numpy()

In [ ]:
X

In [ ]:
if y.to_numpy().shape[1] > 1:
    y = y.iloc[:, 0].to_numpy().flatten()
else:
    y = y.to_numpy().flatten()

In [ ]:
# Fit a random forest model
rf = RandomForestRegressor(n_estimators=100, max_depth=5)
rf.fit(X, y)
rf.score(X, y)

In [ ]:
rf_plus_base = RandomForestPlusRegressor(rf_model=rf)
rf_plus_base.fit(X, y)
rf_plus_base.score(X, y)

In [ ]:
# X, y, _ = imodels.get_clean_dataset("diabetes")
X, y, _ = imodels.get_clean_dataset("diabetes_regr")

In [ ]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
## Debug
# RF Regressor
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
est.fit(X_train, y_train)

# RFplus default(fit on all)
rf_plus_base = RandomForestPlusRegressor(rf_model=est)
rf_plus_base.fit(X_train, y_train)

rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

In [ ]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
temp = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

In [ ]:
temp.shape

In [ ]:
temp[0,0,:].shape

In [ ]:
r2_score([y_train[0]]*100, temp[1,1,:])

In [ ]:
# # RF Regressor
# est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
# est.fit(X_train, y_train)

# # RFplus default(fit on all)
# rf_plus_base = RandomForestPlusRegressor(rf_model=est)
# rf_plus_base.fit(X_train, y_train)

# # RFplus oob 
# rf_plus_base_oob = RandomForestPlusRegressor(rf_model=est, fit_on="oob")
# rf_plus_base_oob.fit(X_train, y_train)

# #RFplus inbag RF
# rf_plus_base_inbag = RandomForestPlusRegressor(rf_model=est, include_raw=False, fit_on="inbag", prediction_model=LinearRegression())
# rf_plus_base_inbag.fit(X_train, y_train)

In [ ]:
# RF Classifier
est = RandomForestClassifier(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
est.fit(X_train, y_train)

# RFplus default(fit on all)
rf_plus_base = RandomForestPlusClassifier(rf_model=est)
rf_plus_base.fit(X_train, y_train)

# RFplus oob 
rf_plus_base_oob = RandomForestPlusClassifier(rf_model=est, fit_on="oob")
rf_plus_base_oob.fit(X_train, y_train)

rf_plus_base_inbag = RandomForestPlusClassifier(rf_model=est, include_raw=False, fit_on="inbag")
rf_plus_base_inbag.fit(X_train, y_train)

# #RFplus inbag RF
# est_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
# est_regressor.fit(X_train, y_train)
# rf_plus_base_inbag = RandomForestPlusRegressor(rf_model=est_regressor, include_raw=False, fit_on="inbag", prediction_model=LinearRegression())
# rf_plus_base_inbag.fit(X_train, y_train)

In [ ]:
# Inbag LMDI+
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

# OOB LMDI+
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

# ALL LMDI+
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

# Inbag LMDI+ l2 norm with sign
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

# OOB LMDI+ l2 norm with sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

# ALL LMDI+ l2 norm with sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

# Inbag LMDI+ l2 norm without sign
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

# OOB LMDI+ l2 norm without sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

# ALL LMDI+ l2 norm without sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

# Inbag LMDI+ with ranking then average
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

# OOB LMDI+ with ranking then average
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

# ALL LMDI+ with ranking then average
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

In [ ]:
# Inbag LMDI+ l2 norm with sign
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

# OOB LMDI+ l2 norm with sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

# ALL LMDI+ l2 norm with sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

In [ ]:
# Inbag LMDI+ l2 norm without sign
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

# OOB LMDI+ l2 norm without sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

# ALL LMDI+ l2 norm without sign
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=False)

In [ ]:
# Inbag LMDI+ with ranking then average
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

# OOB LMDI+ with ranking then average
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

# ALL LMDI+ with ranking then average
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)

In [ ]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

In [ ]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True)

In [ ]:
# RF Classifier
est = RandomForestClassifier(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
est.fit(X_train, y_train)

# RFplus default(fit on all)
rf_plus_base = RandomForestPlusClassifier(rf_model=est)
rf_plus_base.fit(X_train, y_train)

# RFplus oob 
rf_plus_base_oob = RandomForestPlusClassifier(rf_model=est, fit_on="oob")
rf_plus_base_oob.fit(X_train, y_train)

#RFplus inbag RF
est_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
est_regressor.fit(X_train, y_train)
rf_plus_base_inbag = RandomForestPlusRegressor(rf_model=est_regressor, include_raw=False, fit_on="inbag", prediction_model=LinearRegression())
rf_plus_base_inbag.fit(X_train, y_train)

In [ ]:
X_test_pred = est.predict(X_test)
print("R2 score of RF: ", r2_score(y_test, X_test_pred))

In [ ]:
X_test_pred = rf_plus_base.predict(X_test)
print("R2 score of RF+: ", r2_score(y_test, X_test_pred))

In [ ]:
if X_train.shape[0] > 100:
    indices_train = np.random.choice(X_train.shape[0], 100, replace=False)
    X_train_subset = X_train[indices_train]
    y_train_subset = y_train[indices_train]
else:
    indices_train = np.arange(X_train.shape[0])
    X_train_subset = X_train
    y_train_subset = y_train

if X_test.shape[0] > 100:
    indices_test = np.random.choice(X_test.shape[0], 100, replace=False)
    X_test_subset = X_test[indices_test]
    y_test_subset = y_test[indices_test]
else:
    indices_test = np.arange(X_test.shape[0])
    X_test_subset = X_test
    y_test_subset = y_test

In [ ]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="oob")

In [ ]:
rf_plus_mdi.explain(X=X_train, y=y_train)

In [ ]:
temp = rf_plus_mdi.explain(X=X_train, y=y_train)[0][:5]

In [ ]:
temp = np.abs(temp)
temp

In [ ]:
np.argsort(-1*temp)

In [ ]:
np.mean(np.argsort(-1*temp), axis=0)

In [ ]:
rf_plus_mdi.explain(X=X_train, y=y_train)[1][0]

In [ ]:
rf_plus_base.estimators_

In [ ]:
y_train[0]

In [ ]:
treeshap_fi, _, _, _ = tree_shap_evaluation_RF(X_train, y_train, X_train_subset, y_train_subset, X_test, y_test, X_test_subset, y_test_subset, fit=est, mode="absolute", train_only=True)

In [ ]:
lmdi_fi, _, _, _ = LFI_evaluation_RFPlus_oob_l2_norm_sign(X_train, y_train, X_train_subset, y_train_subset, X_test, y_test, X_test_subset, y_test_subset,  fit=rf_plus_base, mode="absolute", train_only=True)

In [ ]:
def select_top_features(array, sorted_indices, percentage):
    array = copy.deepcopy(array)
    num_features = array.shape[1]
    num_selected = int(np.ceil(num_features * percentage))
    selected_indices = sorted_indices[:num_selected]
    selected_array = array[:, selected_indices]
    return num_selected, selected_array

In [ ]:
mask_ratio = [0.05, 0.1, 0.25, 0.5, 0.9]
metric_results_shap_mse = []
metric_results_shap_r2 = []
train_fi_mean = np.mean(treeshap_fi, axis=0)
sorted_feature = np.argsort(-train_fi_mean)
for mask in mask_ratio:
    print(X_train.shape)
    num_features_masked, X_train_masked = select_top_features(X_train, sorted_feature, mask)
    print(X_train_masked.shape)
    num_features_masked, X_test_masked = select_top_features(X_test, sorted_feature, mask)
    print(X_test_masked.shape)
    ablation_models = {"RF_Regressor": RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42)}
                    #"Linear": LinearRegression(),
                    #"XGB_Regressor": xgb.XGBRegressor(random_state=42),
                    # 'Kernel_Ridge': KernelRidge(),
                    #"RF_Plus_Regressor": RandomForestPlusRegressor(rf_model=RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42))}
    # for a_model in ablation_models:
    #     ablation_models[a_model].fit(X_train_masked, y_train)
    rf = LinearRegression()# RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42)
    rf.fit(X_train_masked, y_train)
    y_pred = rf.predict(X_test_masked)
    metric_results_shap_mse.append(mean_squared_error(y_test, y_pred))
    metric_results_shap_r2.append(r2_score(y_test, y_pred))

In [ ]:
select_top_features(X_train, sorted_feature, 0.01)[1][0]

In [ ]:
select_top_features(X_test, sorted_feature, 0.01)[1][0]

In [ ]:
indices = np.nonzero(np.isin(X_train[0], select_top_features(X_train, sorted_feature, 0.01)[1][0]))[0]

print(indices)

In [ ]:
indices = np.nonzero(np.isin(X_test[0], select_top_features(X_test, sorted_feature, 0.01)[1][0]))[0]

print(indices)

In [ ]:
X_train[0]

In [ ]:
metric_results_shap_mse

In [ ]:
# plot metric_results_shap_r2
metric_results_shap_r2 = np.array(metric_results_shap_r2).reshape(len(mask_ratio), -1)
plt.figure()
plt.plot(mask_ratio, metric_results_shap_r2[:, 0], label="RF_Regressor")
plt.xlabel("Feature Ratio")
plt.ylabel("R2")
plt.legend()
plt.show()

In [ ]:
mask_ratio = [0.05, 0.1, 0.25, 0.5, 0.9]
metric_results_lmdi_mse = []
metric_results_lmdi_r2 = []
train_fi_mean = np.mean(local_fi_score_train, axis=0)
sorted_feature = np.argsort(-train_fi_mean)
for mask in mask_ratio:
    print(X_train.shape)
    num_features_masked, X_train_masked = select_top_features(X_train, sorted_feature, mask)
    print(X_train_masked.shape)
    num_features_masked, X_test_masked = select_top_features(X_test, sorted_feature, mask)
    print(X_test_masked.shape)
    ablation_models = {"RF_Regressor": RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42),
                    "Linear": LinearRegression(),
                    "XGB_Regressor": xgb.XGBRegressor(random_state=42),
                    # 'Kernel_Ridge': KernelRidge(),
                    "RF_Plus_Regressor": RandomForestPlusRegressor(rf_model=RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42))}
    for a_model in ablation_models:
        ablation_models[a_model].fit(X_train_masked, y_train)
        y_pred = ablation_models[a_model].predict(X_test_masked)
        metric_results_lmdi_mse.append(mean_squared_error(y_test, y_pred))
        metric_results_lmdi_r2.append(r2_score(y_test, y_pred))